# PedSen Initial ArchR Setup

In [ ]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(ArchR)
quiet_library(hise)
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
#quiet_library(viridis)
#quiet_library(harmony)
#quiet_library(Nebulosa)
quiet_library(stringr)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

# Setup

In [ ]:
ped_arrows <- c('../ped_vs_senior_arrow/B076-P1_PB00124-02.arrow',
                '../ped_vs_senior_arrow/B069-P1_PB00172-02.arrow',
                '../ped_vs_senior_arrow/B069-P1_PB00182-02.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00127-02.arrow',
                '../ped_vs_senior_arrow/B069-P1_PB00807-02.arrow',
                '../ped_vs_senior_arrow/B065-P1_PB00173-02.arrow',
                '../ped_vs_senior_arrow/B065-P1_PB00192-02.arrow',
                '../ped_vs_senior_arrow/B065-P1_PB00197-02.arrow')

In [ ]:
sen_arrows <- c('../ped_vs_senior_arrow/B065-P1_PB00593-04.arrow',
                '../ped_vs_senior_arrow/B069-P1_PB00323-02.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00368-04.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00353-03.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00334-03.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00377-03.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00545-02.arrow',
                '../ped_vs_senior_arrow/B076-P1_PB00599-02.arrow')

## Create ArchR Project

In [ ]:
proj <- ArchRProject(
  ArrowFiles = c(ped_arrows, sen_arrows), 
  outputDirectory = "../PedSen_ATAC",
  copyArrows = TRUE #This is recommened so that if you modify the Arrow files you have an original copy for later usage.
)

In [ ]:
proj

## Subset ArchR Project by barcodes from RNA+ADT object

In [ ]:
rna_adt_so <- readRDS(file = '../Seurat_Objects/sp_merge_only.rds')

In [ ]:
head(proj$cellNames)
clean_barcodes <- str_sub(proj$cellNames, 20, 51)
head(clean_barcodes)

In [ ]:
proj$cellNames_clean <- clean_barcodes

In [ ]:
barcodes_rna <- colnames(rna_adt_so[['RNA']])
head(barcodes_rna)

In [ ]:
idx_new <- which(proj$cellNames_clean %in% barcodes_rna)
head(idx_new)

In [ ]:
length(colnames(rna_adt_so[['RNA']]))
length(proj$cellNames)

In [ ]:
cells_subset <- proj$cellNames[idx_new]
proj_subset <- proj[cells_subset,]
proj_subset

## Transfer Metadata to ArchR Project

In [ ]:
rna_metadata <- rna_adt_so@meta.data
head(rna_metadata)
head(proj_subset$cellNames_clean)

In [ ]:
bc_idx <- barcodes_rna %in% clean_barcodes
table(bc_idx)

In [ ]:
# all cells overlap - so no need to subset rna metadata
# rna_meta_subset <- rna_metadata[bc_idx,]
# length(rownames(rna_meta_subset))

In [ ]:
# check barcode alignment
table(proj_subset$cellNames_clean == rownames(rna_metadata))

In [ ]:
head(rna_metadata)

In [ ]:
rna_metadata <- rna_metadata[match(proj_subset$cellNames_clean, rownames(rna_metadata)),]

In [ ]:
head(rna_metadata)
head(proj_subset$cellNames_clean)

In [ ]:
# check re-aligned barcodes - should be all true
table(proj_subset$cellNames_clean == rownames(rna_metadata))

### Add metadata

In [ ]:
colnames(rna_metadata)

In [ ]:
proj_subset$gating_celltype <- rna_metadata$gating_celltype
proj_subset$pediatric_senior <- rna_metadata$pediatric_senior
proj_subset$CMV <- rna_metadata$CMV
proj_subset$age_cmv <- rna_metadata$age_cmv
proj_subset$predicted.t_celltype.l1 <- rna_metadata$predicted.t_celltype.l1
proj_subset$predicted.t_celltype.l2 <- rna_metadata$predicted.t_celltype.l2
proj_subset$predicted.t_celltype.l3 <- rna_metadata$predicted.t_celltype.l3
proj_subset$batch_id <- rna_metadata$batch_id
proj_subset$pbmc_sample_id <- rna_metadata$pbmc_sample_id

## Add Gene Expression to ArchR Project

In [ ]:
library(SingleCellExperiment)

In [ ]:
rna_mtx <- as.sparse(rna_adt_so@assays$RNA@counts)

In [ ]:
head(colnames(rna_mtx))
head(proj$cellNames)

In [ ]:
archr_style_barcodes <- paste(rna_adt_so$batch_id, "-P1_",rna_adt_so$pbmc_sample_id,"#",colnames(rna_mtx), sep = "")
head(archr_style_barcodes)

In [ ]:
table(archr_style_barcodes %in% proj$cellNames)
rna_adt_so$ATAC_Cell_ID <- archr_style_barcodes

In [ ]:
colnames(rna_mtx) <- archr_style_barcodes

In [ ]:
atac_genes <- getGenes(ArchRProj = proj)
head(atac_genes)

In [ ]:
genes <- getGenes(proj, symbols = rownames(rna_mtx))

In [ ]:
genes

In [ ]:
idx <- rownames(rna_mtx) %in% atac_genes$symbol
head(idx)

In [ ]:
rna_mtx_subset <- rna_mtx[idx,]

In [ ]:
sce <- SingleCellExperiment(list(counts = rna_mtx_subset),rowRanges= genes)
sce

In [ ]:
proj_subset <- addGeneExpressionMatrix(input = proj_subset, seRNA = sce)

# Save ArchR Project

In [ ]:
saveArchRProject(ArchRProj = proj_subset, outputDirectory = '../PedSen_ATAC/', load = F)

In [ ]:
getAvailableMatrices(proj_subset)

In [ ]:
sessionInfo()